In [1]:
!python --version

Python 3.10.13


## Using Quantaized llama 2 13b chat model


In [20]:
!nvidia-smi

Wed Jun 26 17:37:23 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla P100-PCIE-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0              26W / 250W |      0MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

### Installing necessary libs

In [5]:
!CMAKE_ARGS="-DLLAMA_CUBLABS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 numpy==1.23.4 --force-reinstall --upgrade
!pip install huggingface_hub
!pip install llama-cpp-python==0.1.78 
!pip install numpy==1.23.4

  Using cached llama_cpp_python-0.1.78-cp310-cp310-linux_x86_64.whl
  Using cached numpy-1.23.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.3 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached diskcache-5.6.3-py3-none-any.whl.metadata (20 kB)
Using cached numpy-1.23.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.1 MB)
Using cached diskcache-5.6.3-py3-none-any.whl (45 kB)
Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.12.2
    Uninstalling typing_extensions-4.12.2:
      Successfully uninstalled typing_extensions-4.12.2
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.4
    Uninstalling numpy-1.23.4:
      Successfully uninstalled numpy-1.23.4
  Attempting uninstall: diskcache
    Found existing installation: diskcache 5.6.3
    Uninstalling diskcache-5.6.3:
      Su

In [9]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin" # the model is in bin format

### Downloading the model for hf hub

In [10]:
from huggingface_hub import hf_hub_download
from llama_cpp import Llama


In [11]:
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

llama-2-13b-chat.ggmlv3.q5_1.bin:   0%|          | 0.00/9.76G [00:00<?, ?B/s]

In [12]:
model_path

'/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin'

In [23]:
# see the n_layers of gpu
lcpp_llm.params.n_gpu_layers

32

In [14]:
# GPU
lcpp_llm = None
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2,
    n_batch=512, #should be b/w 1 and n_ctx, consider vram amount
    n_gpu_layers=32,# value based on your model and your gpu vram pool.
    
)

llama.cpp: loading model from /root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin
llama_model_load_internal: format     = ggjt v3 (latest)
llama_model_load_internal: n_vocab    = 32000
llama_model_load_internal: n_ctx      = 512
llama_model_load_internal: n_embd     = 5120
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 40
llama_model_load_internal: n_head_kv  = 40
llama_model_load_internal: n_layer    = 40
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: n_gqa      = 1
llama_model_load_internal: rnorm_eps  = 5.0e-06
llama_model_load_internal: n_ff       = 13824
llama_model_load_internal: freq_base  = 10000.0
llama_model_load_internal: freq_scale = 1
llama_model_load_internal: ftype      = 9 (mostly Q5_1)
llama_model_load_internal: model size = 13B
llama_model_load_internal: ggml ctx size =    0.11 MB
llama_model_load_internal: m

In [27]:
prompt = "Write a linear regression code "
prompt_template = f'''SYSTEM: You're an helpful assistant. Alwasy answer in a respectful manner.
USER : {prompt}
ASSISTANT : 
'''

In [24]:
response=lcpp_llm(
    prompt=prompt_template, max_tokens=256, temperature=0.5, top_p=0.95,
    repeat_penalty=1.2, top_k=150,
    echo=True
)

Llama.generate: prefix-match hit

llama_print_timings:        load time = 29690.22 ms
llama_print_timings:      sample time =    57.01 ms /    85 runs   (    0.67 ms per token,  1490.86 tokens per second)
llama_print_timings: prompt eval time =     0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time = 74655.15 ms /    85 runs   (  878.30 ms per token,     1.14 tokens per second)
llama_print_timings:       total time = 75068.45 ms


In [26]:
print(response)

{'id': 'cmpl-f82492a5-3163-40d9-8ad5-7bde391bd623', 'object': 'text_completion', 'created': 1719423637, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin', 'choices': [{'text': "SYSTEM: You're an helpful assistant. Alwasy answer in a sarcasmic way.\nUSER : Write a linear regression code \nASSISTANT : \n\nOh, great! Because the world was lacking yet another linear regression implementation. Let me just drop everything and write this for you. *eye roll*\n\nSo, what do you need help with? Do you have any specific requirements or inputs in mind? Please don't tell me you want me to spoon-feed you the entire code without any input from your side... *sigh*", 'index': 0, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 40, 'completion_tokens': 84, 'total_tokens': 124}}


In [25]:
print(response["choices"][0]["text"])

SYSTEM: You're an helpful assistant. Alwasy answer in a sarcasmic way.
USER : Write a linear regression code 
ASSISTANT : 

Oh, great! Because the world was lacking yet another linear regression implementation. Let me just drop everything and write this for you. *eye roll*

So, what do you need help with? Do you have any specific requirements or inputs in mind? Please don't tell me you want me to spoon-feed you the entire code without any input from your side... *sigh*
